In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/nba'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
#Mostrar que a tabela foi eliminada
spark.sql("""use nba""")

#spark.sql("""show tables""").show()

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

DataFrame[]

In [4]:
# read data from the silver tables
df_players = spark.table("sl_players")
df_teams = spark.table("sl_teams")

In [5]:
df_teams1 = df_teams.select("Ano","Sigla","Conferencia","Lugar","Playoffs","Vitorias%","MelhorJogador")
df_teams1 = df_teams1.withColumnRenamed("Sigla", "Sigla1") \
        .withColumnRenamed("Ano", "Ano1")\
        .withColumnRenamed("Lugar", "LugarConf")

#df_teams1.show()
# join to make a flat table with product information
#JOIN TABELA TEAMS COM PLAYERS
cond = [df_players.Sigla == df_teams1.Sigla1 , df_players.Ano == df_teams1.Ano1]
df_flat = df_players.join(df_teams1,cond)

df_flat = df_flat.drop("Ano1","Sigla1")

#df_flat.show(10000)

In [6]:
from pyspark.sql.functions import substring, avg, sum, max, min, col, when, lit
from pyspark.sql.types import IntegerType, DecimalType

#######
#Estatisticas de um jogador em todas as épocas em que jogou
#######
df_flat = df_flat \
            .withColumn("Count", \
                when((df_flat.Ano.isNotNull()), lit("1").cast('INT')) \
                .otherwise(lit("0").cast('INT')) \
            )\
            .withColumn("Count1", \
                when((df_flat.Playoffs == 'Venceu Final'), lit("1").cast('INT')) \
                .otherwise(lit("0").cast('INT')) \
            )

df0 = df_flat \
    .filter("Liga == 'NBA'") \
    .groupBy("Jogador") \
    .agg(
        sum(df_flat["Count"]).cast('INT').alias("Epocas"),
        sum(df_flat["Count1"]).cast('INT').alias("Campeonatos"),
        avg(df_flat.Minutos).cast(DecimalType(38,2)).alias("MediaMinutosAno"),
        sum(df_flat.Jogos).cast('INT').alias("Jogos"),
        sum(df_flat.Pontos).cast('INT').alias("Pontos") ,
        sum(df_flat.Pontos3P).cast('INT').alias("Pontos3P"),
        sum(df_flat['3P']).cast('INT').alias("Cestos3P"),
        sum(df_flat['3PA']).cast('INT').alias("Tentativas3P"),
        avg(df_flat.Pontos3P).cast(DecimalType(38,2)).alias("MediaPontos3Pano"),
        avg(df_flat['Vitorias%']).cast(DecimalType(38,2)).alias("PercVitorias"),
    )

##df0.where("Jogador" == 'Lebron James').show()

In [ ]:
#1Tabela
spark.sql("drop table if exists nba.gl_players_allyears")

spark.sql(
    """
    CREATE EXTERNAL TABLE nba.gl_players_allyears (
        Jogador STRING,
        Epocas INT,
        Campeonatos INT,
        MediaMinutosAno decimal(38,2),
        Jogos INT,
        Pontos INT, 
        Pontos3P INT,
        Cestos3P INT,
        Tentativas3P INT,
        MediaPontos3Pano decimal(38,2),
        PercVitorias decimal(38,2)
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_players_allyears'
    """
)

df0 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_players_allyears")

#spark.table("nba.gl_players_allyears").show()
#spark.table("nba.gl_players_allyears").count()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/nba/gl_players_allyears/`
""").show()

spark.sql("""
DROP TABLE IF EXISTS nba.gl_players_allyears_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS nba.gl_players_allyears_presto (
        Jogador STRING,
        Epocas INT,
        Campeonatos INT,
        MediaMinutosAno decimal(38,2),
        Jogos INT,
        Pontos INT, 
        Pontos3P INT,
        Cestos3P INT,
        Tentativas3P INT,
        MediaPontos3Pano decimal(38,2),
        PercVitorias decimal(38,2)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_players_allyears/_symlink_format_manifest/'
""").show()

In [7]:
from pyspark.sql.functions import substring, avg, sum, max, min, when, lit
from pyspark.sql.types import IntegerType

########
#Equipas em toda a historia
########

df_aux = df_flat \
    .filter("Liga == 'NBA'") \
    .groupBy("Equipa","Sigla") \
    .agg(
        sum(df_flat.Pontos).cast('INT').alias("Pontos") ,
        sum(df_flat.Pontos3P).cast('INT').alias("Pontos3P"),
        avg(df_flat.Pontos3P).cast(DecimalType(38,2)).alias("MediaPontos3P/Ano"),
    ) \
    .withColumnRenamed("Equipa","Equipa1")\
    .withColumnRenamed("Sigla","Sigla1")

df_teams2 = df_teams \
            .withColumn("Count", \
                when((df_teams.Playoffs == 'Venceu Final'), lit("1").cast('INT')) \
                .otherwise(lit("0").cast('INT')) \
            ) \
            .withColumn("Count1", \
                when((df_teams.Lugar == '1'), lit("1").cast('INT')) \
                .otherwise(lit("0").cast('INT')) \
            ) \
            .withColumn("Count2", \
                when((df_teams.Ano.isNotNull()), lit("1").cast('INT')) \
                .otherwise(lit("0").cast('INT')) \
            )

df1 = df_teams2 \
    .filter("Liga == 'NBA'") \
    .groupBy("Equipa","Sigla","Conferencia") \
    .agg(
        sum(df_teams2["Count2"]).cast('INT').alias("Epocas"),
        avg(df_teams2["Vitorias%"]).cast(DecimalType(38,2)).alias("MediaVitorias"),
        sum(df_teams2["Count"]).cast('INT').alias("Campeonatos"),
        sum(df_teams2["Count1"]).cast('INT').alias("CampeaoConferencia"),
        sum(df_teams2.Jogos).cast('INT').alias("Jogos"),
        avg(df_teams2["ORtg"]).cast(DecimalType(38,2)).alias("MediaORtg"),
        avg(df_teams2["DRtg"]).cast(DecimalType(38,2)).alias("MediaDRtg"),
    )

cond = [df1["Equipa"] == df_aux["Equipa1"] , df1["Sigla"] == df_aux["Sigla1"]]
df1 = df1.join(df_aux,cond)

df1 = df1 \
    .drop("MediaPontos3P/Ano","Equipa1","Sigla1") \
    .withColumn("MediaPontos3PAno", (col("Pontos3P") / col("Epocas")).cast(DecimalType(38,2)))

In [ ]:
#2Tabela
spark.sql("drop table if exists nba.gl_teams_allyears")

spark.sql(
    """
    CREATE EXTERNAL TABLE nba.gl_teams_allyears (
        Equipa STRING,
        Sigla STRING,
        Conferencia STRING,
        Epocas INT,
        MediaVitorias decimal(38,2), 
        Campeonatos INT,
        CampeaoConferencia INT,
        Jogos INT,
        MediaORtg decimal(38,2),
        MediaDRtg decimal(38,2),
        Pontos INT, 
        Pontos3P INT,
        MediaPontos3PAno decimal(38,2)
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_teams_allyears'
    """
)

df1 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_teams_allyears")

#spark.table("nba.gl_teams_allyears").show()
#spark.table("nba.gl_teams_allyears").count()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/nba/gl_teams_allyears/`
""").show()

spark.sql("""
DROP TABLE IF EXISTS nba.gl_teams_allyears_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS nba.gl_teams_allyears_presto (
        Equipa STRING,
        Sigla STRING,
        Conferencia STRING,
        Epocas INT,
        MediaVitorias decimal(38,2), 
        Campeonatos INT,
        CampeaoConferencia INT,
        Jogos INT,
        MediaORtg decimal(38,2),
        MediaDRtg decimal(38,2),
        Pontos INT, 
        Pontos3P INT,
        MediaPontos3PAno decimal(38,2)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_teams_allyears/_symlink_format_manifest/'
""").show()

In [ ]:
##NR TRIPLOS POR CADA ANO POR POSICAO
df3 = df_flat\
    .filter("Liga == 'NBA'") \
    .filter("Ano > '1980'") \
    .groupBy("Posicao","Ano") \
    .agg(
        sum(df_flat["3P"]).cast('INT').alias("NumeroTriplos"),
        sum(df_flat["Pontos3P"]).cast('INT').alias("PontosTriplos"),
    )

#df3.show()

In [ ]:
#3Tabela
spark.sql("drop table if exists nba.gl_three_position")

spark.sql(
    """
    CREATE EXTERNAL TABLE nba.gl_three_position (
        Posicao STRING,
        Ano STRING,
        NumeroTriplos INT,
        PontosTriplos INT
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_three_position'
    """
)

df3 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_three_position")

#spark.table("nba.gl_three_position").show()
#spark.table("nba.gl_three_position").count()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/nba/gl_three_position/`
""").show()

spark.sql("""
DROP TABLE IF EXISTS nba.gl_three_position_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS nba.gl_three_position_presto (
        Posicao STRING,
        Ano STRING,
        NumeroTriplos INT,
        PontosTriplos INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_three_position/_symlink_format_manifest/'
""").show()

In [ ]:
#(Influencia dos Triplos no numero de pontos)
#media de pontos por ano até 1980
#media de pontos por ano apartir de 1980
import pandas as pd

df4 = df_flat \
    .filter("Liga == 'NBA'") \
    .filter("Ano < '1980'") \
    .agg(
        sum(df_flat["Pontos"]).cast('INT').alias("Pontos"),
        min(df_flat["Ano"]).cast('INT').alias("AnoInicio"),
        max(df_flat["Ano"]).cast('INT').alias("AnoFim"),
    ) \
    .withColumn("Altura", lit("Antes1980"))\
    .withColumn("NrAnos", col("AnoFim") - col("AnoInicio"))\
    .withColumn("MediaPontosAno", (col("Pontos") / col("NrAnos")).cast(DecimalType(38,2)) )\
    .drop("AnoInicio","AnoFim")\
    .select("Altura","Pontos","MediaPontosAno","NrAnos")

df5 = df_flat \
    .filter("Liga == 'NBA'") \
    .filter("Ano >= '1980'") \
    .agg(
        sum(df_flat["Pontos"]).cast('INT').alias("Pontos"),
        min(df_flat["Ano"]).cast('INT').alias("AnoInicio"),
        max(df_flat["Ano"]).cast('INT').alias("AnoFim"),
    ) \
    .withColumn("Altura", lit("Depois1980"))\
    .withColumn("NrAnos", col("AnoFim") - col("AnoInicio"))\
    .withColumn("MediaPontosAno", (col("Pontos") / col("NrAnos")).cast(DecimalType(38,2)) )\
    .drop("AnoInicio","AnoFim")\
    .select("Altura","Pontos","MediaPontosAno","NrAnos")

df6 = df4.union(df5)
#df6.show()
#df6.printSchema()

In [ ]:
#4Tabela
spark.sql("drop table if exists nba.gl_after_before")

spark.sql(
    """
    CREATE EXTERNAL TABLE nba.gl_after_before (
        Altura STRING,
        Pontos INT,
        MediaPontosAno decimal(38,2),
        NrAnos INT
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_after_before'
    """
)

df6 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_after_before")

#spark.table("nba.gl_after_before").show()
#spark.table("nba.gl_three_position").count()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/nba/gl_after_before/`
""").show()

spark.sql("""
DROP TABLE IF EXISTS nba.gl_after_before_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS nba.gl_after_before_presto (
        Altura STRING,
        Pontos INT,
        MediaPontosAno decimal(38,2),
        NrAnos INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_after_before/_symlink_format_manifest/'
""").show()

In [ ]:
##jogadores/ano
df_new = df_teams.select("Ano","Sigla","Conferencia","Lugar","Playoffs","Vitorias%","MelhorJogador","Vitorias","Derrotas","Jogos")
df_new = df_new.withColumnRenamed("Sigla", "Sigla1") \
        .withColumnRenamed("Ano", "Ano1")\
        .withColumnRenamed("Lugar", "LugarConf")\
        .withColumnRenamed("Vitorias", "VitoriasEquipa")\
        .withColumnRenamed("Derrotas", "DerrotasEquipa")\
        .withColumnRenamed("Jogos","JogosEquipa")

cond = [df_players.Sigla == df_new.Sigla1 , df_players.Ano == df_new.Ano1]
df_new1 = df_players.join(df_new,cond)

df_new2 = df_new1.drop("Ano1","Sigla1","PER","TS%","3PAR","OBPM","OWS")

df_new3 = df_new2\
    .filter("Liga == 'NBA'")\
    .drop_duplicates()\
    .withColumnRenamed("%Pontos3P","PercPontos3P")\
    .drop("JogosEquipa","VitoriasEquipa","DerrotasEquipa","2P","2PA","LugarConf","Playoffs","Vitorias%")

#df_new3.show()

In [ ]:
#5Tabela
spark.sql("drop table if exists nba.gl_players_year")

spark.sql(
    """
    CREATE EXTERNAL TABLE nba.gl_players_year (
        Ano STRING,
        Equipa STRING,
        Sigla STRING,
        Liga STRING,
        Jogador STRING,
        Idade INT,
        Posicao STRING,
        Jogos INT,
        Minutos INT,
        Pontos INT,
        TotalCestos INT,
        Pontos3P INT,
        3P INT,
        3PA INT,
        PercPontos3P DOUBLE,
        Pontos2P INT,
        WS DOUBLE,
        Conferencia STRING,
        MelhorJogador STRING
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_players_year'
    """
)

df_new3 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_players_year")

#spark.table("nba.gl_players_year").show()
#spark.table("nba.gl_three_position").count()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/nba/gl_players_year/`
""").show()

spark.sql("""
DROP TABLE IF EXISTS nba.gl_players_year_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS nba.gl_players_year_presto (
        Ano STRING,
        Equipa STRING,
        Sigla STRING,
        Liga STRING,
        Jogador STRING,
        Idade INT,
        Posicao STRING,
        Jogos INT,
        Minutos INT,
        Pontos INT,
        TotalCestos INT,
        Pontos3P INT,
        3P INT,
        3PA INT,
        PercPontos3P DOUBLE,
        Pontos2P INT,
        WS DOUBLE,
        Conferencia STRING,
        MelhorJogador STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_players_year/_symlink_format_manifest/'
""").show()

In [ ]:
#####
#equipas/ano
#####
df_newteams = df_new2\
    .filter("Liga == 'NBA'") \
    .groupBy("Ano","Equipa","Sigla","Conferencia","LugarConf","Playoffs","MelhorJogador","VitoriasEquipa","DerrotasEquipa","JogosEquipa",col("Vitorias%"))\
    .agg(
        sum(df_new2["Pontos"]).cast('INT').alias("Pontos"),
        sum(df_new2["Pontos3P"]).cast('INT').alias("PontosTriplos"),
    )\
    .withColumnRenamed("Vitorias%","PercVitorias")

#df_newteams.where(col("Equipa") == "Boston Celtics").show()

In [ ]:
#6Tabela
spark.sql("drop table if exists nba.gl_teams_year")

spark.sql(
    """
    CREATE EXTERNAL TABLE nba.gl_teams_year (
        Ano STRING,
        Equipa STRING,
        Sigla STRING,
        Conferencia STRING,
        LugarConf STRING,
        Playoffs STRING,
        MelhorJogador STRING,
        VitoriasEquipa INT,
        DerrotasEquipa INT,
        JogosEquipa INT,
        PercVitorias DOUBLE,
        Pontos INT,
        PontosTriplos INT
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year'
    """
)

df_newteams \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year")

#spark.table("nba.gl_teams_year").show()
#spark.table("nba.gl_three_position").count()

In [ ]:
df_newteams \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year")

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year/`
""").show()

spark.sql("""
DROP TABLE IF EXISTS nba.gl_teams_year_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS nba.gl_teams_year_presto (
        Ano STRING,
        Equipa STRING,
        Sigla STRING,
        Conferencia STRING,
        LugarConf STRING,
        Playoffs STRING,
        MelhorJogador STRING,
        VitoriasEquipa INT,
        DerrotasEquipa INT,
        JogosEquipa INT,
        PercVitorias DOUBLE,
        Pontos INT,
        PontosTriplos INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year/_symlink_format_manifest/'
""").show()

In [ ]:
from pyspark.sql.functions import substring, avg, sum, max, min, when, lit
adc = spark.table("gl_teams_year")
adc1 = adc \
    .withColumn("Pais", lit("United States of America"))\
    .withColumn("Conferencia", \
                when((adc.Conferencia == 'Este'), lit("East")) \
                .when((adc.Conferencia == 'Oeste'), lit("West")) \
                .otherwise("NaN") \
               )

adc1 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year")

#spark.table("nba.gl_teams_year").show()

In [ ]:
spark.sql("use nba")
adc2 = spark.table("gl_teams_year")

In [ ]:
#adc3 = adc2.filter("State == 'Louisiana'").show()

In [ ]:
from pyspark.sql.functions import substring, avg, sum, max, min, col, when, lit
adc3 = adc2 \
    .withColumn("State", \
                when((adc2.Equipa == 'Phoenix Suns'), lit("Arizona"))\
                .when((adc2.Equipa == 'Golden State Warriors'), lit("California"))\
                .when((adc2.Equipa == 'Los Angeles Clippers'), lit("California"))\
                .when((adc2.Equipa == 'Los Angeles Lakers'), lit("California"))\
                .when((adc2.Equipa == 'Sacramento Kings'), lit("California"))\
                .when((adc2.Equipa == 'Denver Nuggets'), lit("Colorado"))\
                .when((adc2.Equipa == 'Miami Heat'), lit("Florida"))\
                .when((adc2.Equipa == 'Orlando Magic'), lit("Florida"))\
                .when((adc2.Equipa == 'Atlanta Hawks'), lit("Georgia"))\
                .when((adc2.Equipa == 'Chicago Bulls'), lit("Illinois"))\
                .when((adc2.Equipa == 'Indiana Pacers'), lit("Indiana"))\
                .when((adc2.Equipa == 'New Orleans Pelicans'), lit("Louisiana"))\
                .when((adc2.Equipa == 'Boston Celtics'), lit("Massachusetts"))\
                .when((adc2.Equipa == 'Detroit Pistons'), lit("Michigan"))\
                .when((adc2.Equipa == 'Minnesota Timberwolves'), lit("Minnesota"))\
                .when((adc2.Equipa == 'Brooklyn Nets'), lit("New York"))\
                .when((adc2.Equipa == 'New York Knicks'), lit("New York"))\
                .when((adc2.Equipa == 'New York Nets'), lit("New York"))\
                .when((adc2.Equipa == 'Charlotte Hornets'), lit("North Carolina"))\
                .when((adc2.Equipa == 'Cleveland Cavaliers'), lit("Ohio"))\
                .when((adc2.Equipa == 'Oklahoma City Thunder'), lit("Oklahoma"))\
                .when((adc2.Equipa == 'Portland Trailblazers'), lit("Oregon"))\
                .when((adc2.Equipa == 'Philadelphia 76ers'), lit("Pennsylvania"))\
                .when((adc2.Equipa == 'Memphis Grizzlies'), lit("Tennessee"))\
                .when((adc2.Equipa == 'Dallas Mavericks'), lit("Texas"))\
                .when((adc2.Equipa == 'Houston Rockets'), lit("Texas"))\
                .when((adc2.Equipa == 'San Antonio Spurs'), lit("Texas"))\
                .when((adc2.Equipa == 'Utah Jazz'), lit("Utah"))\
                .when((adc2.Equipa == 'Milwaukee Bucks'), lit("Wisconsin"))\
                .when((adc2.Equipa == 'Washington Wizards'), lit("Maryland"))\
                .when((adc2.Equipa == 'Toronto Raptors'), lit("New York"))\
                .when((adc2.Equipa == 'Seattle Supersonics'), lit("Washington"))\
                .when((adc2.Equipa == 'New York Knickerbockers'), lit("New York"))\
                .when((adc2.Equipa == 'Buffalo Braves'), lit("California"))\
                .when((adc2.Equipa == 'New Jersey Nets'), lit("New York"))\
                .when((adc2.Equipa == 'Denver Rockets/Nuggets'), lit("Colorado"))\
                .when((adc2.Equipa == 'New Orleans/Oklahoma City Hornets'), lit("Louisiana"))\
                .when((adc2.Equipa == 'St. Louis Hawks'), lit("Georgia"))\
                .when((adc2.Equipa == 'Syracuse Nationals'), lit("Pennsylvania"))\
                .when((adc2.Equipa == 'Baltimore Bullets'), lit("Maryland"))\
                .when((adc2.Equipa == 'San Francisco Warriors'), lit("California"))\
                .when((adc2.Equipa == 'Cincinnati Royals'), lit("Kansas"))\
                .when((adc2.Equipa == 'Milwaukee Hawks'), lit("Wisconsin"))\
                .when((adc2.Equipa == 'Rochester Royals'), lit("California"))\
                .when((adc2.Equipa == 'Kansas City-Omaha Kings'), lit("Kansas"))\
                .when((adc2.Equipa == 'Kansas City Kings'), lit("Kansas"))\
                .when((adc2.Equipa == 'Philadelphia Warriors'), lit("Pennsylvania"))\
                .when((adc2.Equipa == 'Fort Wayne Zollner Pistons/Fort Wayne Pistons'), lit("Indiana"))\
                .when((adc2.Equipa == 'San Diego Clippers'), lit("California"))\
                .when((adc2.Equipa == 'Vancouver Grizzlies'), lit("Tennessee"))\
                .when((adc2.Equipa == 'San Diego Rockets'), lit("California"))
               )\
        .withColumn("Conferencia", when((adc2.Equipa == 'Buffalo Braves'), lit("West"))\
                    .when((adc2.Equipa == 'Vancouver Grizzlies'), lit("East"))\
                    .when((adc2.Equipa == 'New Orleans/Oklahoma City Hornets'), lit("East"))\
                    .otherwise(col("Conferencia")))

#adc4 = adc3.filter("State == 'California'").show()
#adc3.show()

adc3 \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year")

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year/`
""").show()

spark.sql("""
DROP TABLE IF EXISTS nba.gl_teams_year_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS nba.gl_teams_year_presto (
        Ano STRING,
        Equipa STRING,
        Sigla STRING,
        Conferencia STRING,
        LugarConf STRING,
        Playoffs STRING,
        MelhorJogador STRING,
        VitoriasEquipa INT,
        DerrotasEquipa INT,
        JogosEquipa INT,
        PercVitorias DOUBLE,
        Pontos INT,
        PontosTriplos INT,
        Pais STRING,
        State STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/gl_teams_year/_symlink_format_manifest/'
""").show()